Paul O'Leary

Comp 4448

MiniProject - Assignment 5 - Naive Bayes

July 26, 2021



In [1]:
# Load some of the standards

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from scipy import stats


In [ ]:
# =============================================
# ---
# =============================================

4) Mini Project
Find some text data of your own choice, it could be labelled tweets, etc. 
Your dataset should have at least 200 instances, and if there are several columns of text, you can choose to merge the text columns into a single text column. Each text instance should have at least 60 words. 
Clean the data, split the data, transform the data to a representation suitable for your algorithm, build your model and evaluate the model. Tune some parameters of interest and write a short report about what problem your mini project is trying to address, the description of your data, the choice of algorithm used, the performance of your algorithm, overfitting, the choice of hyperparameters tunned, then your recommendation or conclusion (imagine you were trying to recommend this algorithm to a stakeholder, and you need this report to include important and persuasive elements). Your report could be in one or two paragraphs and should include relevant code and output at the end.

In [60]:
# !!!?!!!

Can a wine reviewer's description predict if the wine will be rated at more than 92 points?

Dataset:  https://www.kaggle.com/zynicide/wine-reviews?select=winemag-data-130k-v2.csv.  "Wine Reviews: 130k wine reviews with variety, location, winery, price, and description"

In [61]:
# ===========================

A certain member of my household (my wife) is a big fan of fine wines, carefully reads wine reviews from a variety of sources and frequently bases her purchases on the rating points assigned.  The descriptions accompanying the ratings frequently use language that only a wine drinker would understand.  Can the words used by a reviewer to describe a wine accurately predict the rating points that it recieves?


This dataset is made up of data from Wine Magazine, compiled by "zackthoutt" and made available via "Kaggle". The full dataset has over 130,000 records with 25 columns of information.  For this 'mini-project' 300 records with 60+ words in the description were pulled, along with the points assigned, regardless of wine variety, region, winery, points, etc.  After the records were pulled from the file, the rating points were binned into "less than 92" rating (False) and "greater than or equal to 92" rating (True).  The description field was cleaned of stop words and punctuation, and the tfidfVectorizer was then used to prep the data.  A multinomial Naive Bayes classifier was used on the vectorized words to determine if the description could predict ratings greater than or less than 92 points.

With the default settings for the vectorizer and for the multinomialNB, the Accuracy score was 80% for the training data and 70% for the test data - meaning the model predicted the correct rating range 76% of the time on the undeen data, which is much better than a coin flip. (NOTE:  Each time the cells below are rerun, the results change slightly due to the small number of records being used, and the 70/30 train/test split.  Apologies if the numbers are slightly different in the code.)  After hyper-tuning parameters max_features, min_df': 5, ngram_range' and use_idf, the results improved to 83% accuracy for the test data.

Given the small amount of data used, these results are quite promising.  Wine ratings are highly subjective, but the use of a multinomial Naive Bayes model approach to predict ratings performed well.  The next step would be to attempt to predict the actual numeric points rating with a regression model with a much larger set of data.  

All the code and analysis is below:

In [62]:
# ===========================

In [63]:
# Large file!  

wr = pd.read_csv("winemag-data_first150k.csv")

In [64]:
wr.head(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [66]:
wr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   150930 non-null  int64  
 1   country      150925 non-null  object 
 2   description  150930 non-null  object 
 3   designation  105195 non-null  object 
 4   points       150930 non-null  int64  
 5   price        137235 non-null  float64
 6   province     150925 non-null  object 
 7   region_1     125870 non-null  object 
 8   region_2     60953 non-null   object 
 9   variety      150930 non-null  object 
 10  winery       150930 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 12.7+ MB


In [67]:
gr60 = wr[wr.description.str.count(' ') > 60]  # Should pull descriptions with more than 60 words

In [68]:
gr60.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6652 entries, 3 to 150927
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   6652 non-null   int64  
 1   country      6651 non-null   object 
 2   description  6652 non-null   object 
 3   designation  5156 non-null   object 
 4   points       6652 non-null   int64  
 5   price        6515 non-null   float64
 6   province     6651 non-null   object 
 7   region_1     5854 non-null   object 
 8   region_2     3936 non-null   object 
 9   variety      6652 non-null   object 
 10  winery       6652 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 623.6+ KB


In [69]:
# Still 6600+ records

In [70]:
gr60.head(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
14,14,US,"With its sophisticated mix of mineral, acid an...",Grace Vineyard,95,185.0,Oregon,Dundee Hills,Willamette Valley,Pinot Noir,Domaine Serene
18,18,France,Coming from a seven-acre vineyard named after ...,Le Pigeonnier,95,290.0,Southwest France,Cahors,NaN,Malbec,Château Lagrézette
22,22,Spain,Tarry blackberry and cheesy oak aromas are app...,Termanthia,95,220.0,Northern Spain,Toro,NaN,Tinta de Toro,Numanthia


In [71]:
wr_small = gr60.iloc[:300]
wr_small = wr_small[['description', 'points']]

In [72]:
wr_small

,description,points
3,"This spent 20 months in 30% new French oak, an...",96
4,"This is the top wine from La Bégude, named aft...",95
14,"With its sophisticated mix of mineral, acid an...",95
18,Coming from a seven-acre vineyard named after ...,95
22,Tarry blackberry and cheesy oak aromas are app...,95
...,...,...
10800,This is a potent brew with complexities galore...,94
10801,"Amazingly rich and voluptuous, this shows none...",94
10803,The last Altos that was this good came in 2004...,94
10805,The “X” designation refers to the number of cl...,94


In [73]:
wr_small.describe()

,points
count,300.000000
mean,92.600000
std,2.612038
min,85.000000
25%,91.000000
50%,93.000000
75%,94.000000
max,99.000000


In [74]:
# This may be difficult since these wines are all fairly highly rated, with the average points at 92.6 points, and with the small number of records.

In [75]:
# double check the counts of the number of words in the description

count = wr_small['description'].str.split().apply(len).value_counts()
# count.index = count.index.astype(str) + ' words:'
# # count.sort_index(inplace=True)
# count.head(50)

count

62     43
65     32
63     31
64     31
66     22
68     20
69     16
70     14
67     13
71     10
79      8
75      6
74      6
73      5
83      5
72      5
80      3
82      3
84      3
78      3
77      3
76      3
87      2
89      2
92      2
81      1
86      1
90      1
94      1
97      1
104     1
110     1
116     1
120     1
Name: description, dtype: int64

In [76]:
# Looks OK

In [77]:
# Now I need to split the reviews into < 92 and >= 92 buckets.  I do it with Bools - 0 for false, 1 for true

wr_small["Bool_rate"] = (wr_small['points'] >= 92).astype(int)

In [78]:
wr_small.head(10)

,description,points,Bool_rate
3,"This spent 20 months in 30% new French oak, an...",96,1
4,"This is the top wine from La Bégude, named aft...",95,1
14,"With its sophisticated mix of mineral, acid an...",95,1
18,Coming from a seven-acre vineyard named after ...,95,1
22,Tarry blackberry and cheesy oak aromas are app...,95,1
141,A sublime twist of citrus lies at the heart of...,90,0
161,A blend of fruit from three Rogue Valley sites...,89,0
227,"Chris Gorman is the winemaker for this pure, o...",93,1
230,"Somewhat light in color, this is an aromatical...",93,1
293,"This is a gigantic Syrah, a tsunami of blackbe...",91,0


In [79]:
# Proceed with some analysis.  I'll use the "clean" function from above.

In [80]:
wr_small["description"] = wr_small["description"].apply(clean, stopwords = sw)

In [81]:
wr_small.head()

,description,points,Bool_rate
3,spent 20 months 30 new french oak incorporates...,96,1
4,top wine la bégude named highest point vineyar...,95,1
14,sophisticated mix mineral acid tart fruits sed...,95,1
18,coming seven-acre vineyard named dovecote prop...,95,1
22,tarry blackberry cheesy oak aromas appropriate...,95,1


In [82]:
# Split the data

X_train, X_test, y_train, y_test=train_test_split(wr_small["description"], wr_small["Bool_rate"], test_size=0.3)

In [83]:
tfidf=TfidfVectorizer(ngram_range=(1,2), stop_words="english", min_df=10, max_features=None)
X_train_tfidf=tfidf.fit_transform(X_train)

In [84]:
clf=MultinomialNB()
clf=clf.fit(X_train_tfidf, y_train)
clf.predict(X_train_tfidf)

array([1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [85]:
clf.score(X_train_tfidf, y_train)

0.8047619047619048

In [86]:
y_pred_train = clf.predict(X_train_tfidf)
print("Accuracy on Train set: ", accuracy_score(y_train, y_pred_train))

Accuracy on Train set:  0.8047619047619048


In [87]:
# 80%. On the Test data?

X_test_tfidf=tfidf.fit_transform(X_test)
clf=clf.fit(X_test_tfidf, y_test)
y_pred_test = clf.predict(X_test_tfidf)
print("Accuracy on Test set: ", accuracy_score(y_test, y_pred_test))

Accuracy on Test set:  0.7


In [88]:
# 70% accurate results.  I'm actually fairly impressed that it can be that accurate, given the quite subjective topic.

In [89]:
# Try some parameter tuning.

pipe = Pipeline([("tfidf",TfidfVectorizer(stop_words = "english")), ("nb", MultinomialNB())])
param_grid = [{"tfidf__min_df":[5, 20], 
               "tfidf__ngram_range":[(1, 1), (1, 2), (1, 3), (1, 5), (1, 7)],
               "tfidf__use_idf": (True, False),  
               "tfidf__max_features": (None, 5000, 10000)}]
grid = GridSearchCV(estimator = pipe, param_grid = param_grid, cv = 5, n_jobs = -1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('nb', MultinomialNB())]),
             n_jobs=-1,
             param_grid=[{'tfidf__max_features': (None, 5000, 10000),
                          'tfidf__min_df': [5, 20],
                          'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 5),
                                                 (1, 7)],
                          'tfidf__use_idf': (True, False)}])

In [90]:
grid.best_params_

{'tfidf__max_features': None,
 'tfidf__min_df': 5,
 'tfidf__ngram_range': (1, 2),
 'tfidf__use_idf': True}

In [91]:
# See if that helps.  max_features: None is a default, as is use_idf: True

In [92]:
tfidf=TfidfVectorizer(ngram_range=(1,2), stop_words="english", min_df=5, max_features=None)
X_train_tfidf=tfidf.fit_transform(X_train)

clf=MultinomialNB()
clf=clf.fit(X_train_tfidf, y_train)
clf.predict(X_train_tfidf)

clf.score(X_train_tfidf, y_train)

0.861904761904762

In [93]:
# and the Test data?

X_test_tfidf=tfidf.fit_transform(X_test)
clf=clf.fit(X_test_tfidf, y_test)
y_pred_test = clf.predict(X_test_tfidf)
print("Accuracy on Test set: ", accuracy_score(y_test, y_pred_test))

Accuracy on Test set:  0.8333333333333334


In [94]:
# Well, look at that.  Tuning helped the results on both the Training and the Test data.

Tuned Results. 86% accuracy on the Training data, and 83% accuracy on the Test data.


NOTE!!!  Since the dataset is small, each time I reran the Mini-project cells, I got slightly different results, based on the train data selected - including on the hyper-tuned params.  (Specifically, ngram_range changed between (1,2) and (1,3).)  Apologies if the numbers are a bit different.
